In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, roc_auc_score

In [2]:
df1 = input("Enter the new data with prediction: ")
print(f"The file: {df1}")
df1 = pd.read_csv(df1)
print(df1.shape)



The file: data prep.csv
(523, 13)


In [ ]:
df2 = input("Enter the old data: ")  #Loan data.csv
df2 = pd.read_csv(df2)
df = pd.concat([df2,df1])


jk=df.copy()
df.shape


(1110, 13)

In [5]:
df['Loan_Status']=df['Loan_Status'].replace({'Y':1,'N':0}).astype(int)

In [6]:
for col in ['Gender', 'Married', 'Dependents', 'Self_Employed']:
    df[col] = df[col].fillna(df[col].mode(dropna=True)[0])

df['LoanAmount'] = df['LoanAmount'].fillna(df['LoanAmount'].median())
df['Loan_Amount_Term']=df['Loan_Amount_Term'].fillna(df['Loan_Amount_Term'].mode(dropna=True)[0])
df['Credit_History']=df['Credit_History'].fillna(df['Credit_History'].mode(dropna=True)[0])

In [7]:
df['Gender']=np.where(df['Gender'] == 'Male',1,0)
df['Married']=np.where(df['Married'] == 'Yes',1,0 )
df['Dependents']=df['Dependents'].replace('3+',3).astype(int)
df['Education']=np.where(df['Education'] == 'Graduate',1,0)
df['Self_Employed']=np.where(df['Self_Employed'] == 'Yes',1,0)
df['Property_Area']=np.where(df['Property_Area'] == 'Urban',1,0)


In [8]:
# Calculate the IQR
Q1 = df['ApplicantIncome'].quantile(0.25)
Q3 = df['ApplicantIncome'].quantile(0.75)
IQR = Q3 - Q1

# Define the lower and upper bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Remove outliers
df = df[(df['ApplicantIncome'] >= lower_bound) & (df['ApplicantIncome'] <= upper_bound)]

In [9]:
# Calculate the IQR
Q1 = df['CoapplicantIncome'].quantile(0.25)
Q3 = df['CoapplicantIncome'].quantile(0.75)
IQR = Q3 - Q1

# Define the lower and upper bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Remove outliers
df = df[(df['CoapplicantIncome'] >= lower_bound) & (df['CoapplicantIncome'] <= upper_bound)]

In [11]:
y=df['Loan_Status']
x=df[['Credit_History','ApplicantIncome','LoanAmount','CoapplicantIncome','Dependents','Loan_Amount_Term']]

In [12]:
parameters = {"max_depth": [2,5,10,20,50],
              "min_samples_leaf": [2,5,10,20,50],
              "n_estimators": [2,5,10,20,50]}
model = RandomForestClassifier(max_features='log2', random_state=0,class_weight="balanced")
grid_search = GridSearchCV(estimator=model, param_grid=parameters,cv=3, n_jobs=6)
grid_search = grid_search.fit(x,y)
print(grid_search.best_score_)
print(grid_search.best_params_)



0.9521502518106932
{'max_depth': 50, 'min_samples_leaf': 2, 'n_estimators': 50}


In [13]:
md = grid_search.best_params_['max_depth']
ml = grid_search.best_params_['min_samples_leaf']
ne = grid_search.best_params_['n_estimators']

In [15]:
from datetime import datetime
gb = pd.DataFrame({'obs':[x.shape[0]], 'max_depth':[md], 'min_samples_leaf':[ml], 'n_estimators':[ne]}).T
gb.to_csv('Loan_Model hyp '+datetime.today().strftime('%Y-%m-%d')+'.csv', header=None)

In [19]:
import joblib
model = RandomForestClassifier(max_depth=md, min_samples_leaf=ml, n_estimators=ne, 
                              max_features='log2', random_state=0,class_weight="balanced")
model.fit(x,y)
joblib.dump(model,'(S)Loan_Model rf'+datetime.today().strftime('%Y-%m-%d')+'.pkl') 

['(S)Loan_Model rf2025-07-31.pkl']

In [17]:
jk.to_csv('Loan_Model data'+datetime.today().strftime('%Y-%m-%d')+'.csv', index=False)
jk.shape

(1110, 13)